In [73]:
# -*- coding: utf-8 -*-

import openpyxl
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from numpy.linalg import inv
from scipy.stats import norm


def QQ(vec):
    v = np.sort(vec)
    q = stats.norm.ppf((np.arange(len(v))+1-0.375)/(len(v)+0.25))
    return v, q

def RB_fit(RB):
    #RB_raw = np.array(RB)
    RB_ord = np.sort(RB)/100
    th_q = stats.norm.ppf((np.arange(len(RB))+1-0.375)/(len(RB)+0.25))
    q = th_q
    l = np.log(RB_ord)
    h = np.log(1-RB_ord)
    d = (np.mean(l*l) - pow(np.mean(l),2))*(np.mean(h*h) - pow(np.mean(h),2)) - pow(np.mean(l*h) - np.mean(l)*np.mean(h),2)
    d1 = (np.mean(h*h) - pow(np.mean(h),2))*(np.mean(l*q) - np.mean(l)*np.mean(q))-(np.mean(l*h) - np.mean(l)*np.mean(h))*(np.mean(h*q) - np.mean(h)*np.mean(q))
    d2 = (np.mean(l*h) - np.mean(l)*np.mean(h))*(np.mean(l*q) - np.mean(l)*np.mean(q)) - (np.mean(l*l) - pow(np.mean(l),2))*(np.mean(h*q) - np.mean(h)*np.mean(q))
    if d1 < 0:
        a = 0
        b = -(np.mean(h*q) - np.mean(h)*np.mean(q))/(np.mean(h*h) - pow(np.mean(h),2))
    elif d2 < 0:
        a = (np.mean(l*q) - np.mean(l)*np.mean(q))/(np.mean(l*l) - pow(np.mean(l),2))
        b = 0
    else:
        a = d1/d
        b = d2/d
#     a = d1/d
#     b = d2/d
    c = np.mean(q) - a*np.mean(l) + b*np.mean(h)
    RB_t = a*np.log(RB/100) - b*np.log(1-RB/100) + c
    _, pv = stats.shapiro(RB_t)
    return a, b, c, pv, RB_t, th_q

def TP_fit(TP):
    TP_ord = np.sort(TP)
    th_q = stats.norm.ppf((np.arange(len(TP))+1-0.375)/(len(TP)+0.25))
    q = th_q
    g = np.log(TP_ord)
    t = TP_ord
    d = (np.mean(g*g) - pow(np.mean(g),2))*(np.mean(t*t) - pow(np.mean(t),2)) - pow(np.mean(g*t) - np.mean(g)*np.mean(t),2)
    d1 = (np.mean(t*t) - pow(np.mean(t),2))*(np.mean(g*q) - np.mean(g)*np.mean(q))-(np.mean(g*t) - np.mean(g)*np.mean(t))*(np.mean(t*q) - np.mean(t)*np.mean(q))
    d2 = (np.mean(g*g) - pow(np.mean(g),2))*(np.mean(t*q) - np.mean(t)*np.mean(q))-(np.mean(t*g) - np.mean(t)*np.mean(g))*(np.mean(g*q) - np.mean(g)*np.mean(q))
    if d1 < 0:
        a = 0
        b = (np.mean(t*q) - np.mean(t)*np.mean(q))/(np.mean(t*t) - pow(np.mean(t),2))
    elif d2 < 0:
        a = (np.mean(g*q) - np.mean(g)*np.mean(q))/(np.mean(g*g) - pow(np.mean(g),2))
        b = 0
    else:
        a = d1/d
        b = d2/d
#     a = d1/d
#     b = d2/d
    c = np.mean(q) - a*np.mean(g) - b*np.mean(t)
    TP_t = a*np.log(TP) + b*TP + c
    _, pv = stats.shapiro(TP_t)
    return a, b, c, pv, TP_t, th_q

RD = openpyxl.load_workbook('data_prep_ver2.xlsx', data_only=True)
st = RD['Sheet2']   #band 20MHz
#st = RD['Sheet1']   #band 10MHz

Data = np.zeros((1,5))
i = 2
value = st['B%d' %i].value
while value != None:
    Data = np.vstack((Data, np.array([st['B%d' %i].value, st['C%d' % i].value, st['D%d' %i].value, st['E%d' %i].value, st['F%d' %i].value])))
    # PCI, RB, RNTI, TP, Timestamp
    i += 1
    value = st['B%d' %i].value
Data = Data[1:,:].astype(np.float64)

In [74]:
#length가 30 미만인 RNTI 삭제
len_per_rnti = np.array([len(Data[Data[:,2] == np.float(i), 1]) for i in range(20)])
selected_rnti = np.where(len_per_rnti>30)
Data = Data[np.where(np.isin(Data[:,2], selected_rnti)),]
Data = np.reshape(Data, (len(Data[0]),5))

In [75]:
def Roystontest(n, k, Data):
    x = np.log(n)
    gamma = 0
    mu = -1.5861 - 0.31082*x - 0.083751*pow(x,2) + 0.0038915*pow(x,3)
    sigma = np.exp(-0.4803 -0.082676*x + 0.0030302*pow(x,2))
    W1 = stats.shapiro(Data[:,0])[0]
    W2 = stats.shapiro(Data[:,1])[0]
    Z1 = (np.log(1-W1)-mu)/sigma   
    Z2 = (np.log(1-W2)-mu)/sigma
    G1 = stats.norm.ppf(stats.norm.cdf(-Z1)/2)**2
    G2 = stats.norm.ppf(stats.norm.cdf(-Z2)/2)**2
    u = 0.715
    v = 0.21364+0.015124*pow(x,2)-0.0018034*pow(x,3)
    la = 5
    corr_data = np.corrcoef(Data[:,0],Data[:,1])
    new_corr = pow(corr_data,la) * (1 - u*pow(1 - corr_data,u)/v)
    total_corr = np.sum(new_corr) - k
    avg_corr = total_corr/(k**2-k)
    est_e = k/(1 + (k - 1) * avg_corr)
    Roys = est_e*(G1+G2)/k
    p_value = 1- stats.chi2.cdf(Roys, df = est_e)
    return p_value

In [76]:
selected_rnti = np.array(selected_rnti)
selected_rnti = selected_rnti.reshape(-1)

In [77]:
def rntiprob(length):
    rntiprob = np.zeros(length)   #rnti의 비율 계산
    rntiprobsum = np.zeros(length)   # rnti 비율 누적
    for i in range(length):
        rntiprob[i] = len(Data[Data[:,2] == (i+3),2])/len(Data[:,2])
    for i in range(length):
        rntiprobsum[i] = sum(rntiprob[0:i+1])
    return rntiprobsum

rntiprobsum = rntiprob(len(selected_rnti))
def fixRNTI(rntiprobsum):
    a = np.random.uniform(0,1)
    i = 0
    while rntiprobsum[i] < a:
        i = i+1
    return i+3

#fixRNTI(rntiprob(len(selected_rnti)))

In [79]:
def SamplingperRnti(Data, fixed_RNTI):
    RB_sample = Data[Data[:,2] == fixed_RNTI, 1]   # RNTI = N 인 RB,RNTI,TP 
    TP_sample = Data[Data[:,2] == fixed_RNTI, 3]
    sampling_size = min(30, int(len(RB_sample)/2))
    sample_index = np.random.choice(len(RB_sample),sampling_size, replace = False)
    RB_sample = RB_sample[sample_index,]
    TP_sample = TP_sample[sample_index,]
    return(RB_sample, TP_sample)

def FindParams(fixed_rnti, Data):
    RB_rnd_sample, TP_rnd_sample = SamplingperRnti(Data, fixed_rnti)   # random한 RNTI에 대해 RB, TP random sampling
    a_RB, b_RB, c_RB, pv_RB, RB_trans_data, tq_RB = RB_fit(RB_rnd_sample)
    a_TP, b_TP, c_TP, pv_TP, TP_trans_data, tq_TP = TP_fit(TP_rnd_sample)
    total_trans_data = np.hstack((RB_trans_data.reshape(-1,1), TP_trans_data.reshape(-1,1)))   # transformed data로 total data 생성
    cov = np.cov(total_trans_data.T)[0,1]
    #corr = np.corrcoef(total_trans_data.T)[0,1]
    joint_pv_R = Roystontest(len(TP_trans_data), 2, total_trans_data)
    se = np.mean(TP_rnd_sample*0.001/(RB_rnd_sample*2/100*10))
    if (pv_RB > 0.05)& (pv_TP >0.05) & (joint_pv_R > 0.05):
        return np.array([fixed_rnti, a_RB, b_RB, c_RB, a_TP, b_TP, c_TP, np.mean(total_trans_data), cov, se])
    else:
        return [0]
    
ParamArray = np.zeros((9))
sample_size =10000

i = 0
transtp10 = []
while i < sample_size:
    fixed_rnti = fixRNTI(rntiprobsum)
    #fixed_rnti = selected_rnti[np.random.randint(len(selected_rnti))]
    params = FindParams(fixed_rnti, Data)
    if len(params) == 9:
        ParamArray = np.vstack((ParamArray, params))
        i += 1
    else:
        continue
    
print("end")

end


In [82]:
raw_data_20 = ParamArray[1:,:]

In [63]:
##### sample mean; 10MHz
sample_mean_10MHz = np.zeros(9)
for i in range(9):
    sample_mean_10MHz[i] = np.mean(raw_data_10[np.where(np.isin(raw_data_10[:,0], i+3)),8])

In [83]:
##### sample mean; 20MHz
sample_mean_20MHz = np.zeros(16)
for i in range(16):
    sample_mean_20MHz[i] = np.mean(raw_data_20[np.where(np.isin(raw_data_20[:,0], i+3)),8])
sample_mean_20MHz

array([0.88663964, 1.10382795, 1.14001456, 1.29547738, 1.38441841,
       1.42375755, 1.40577229, 1.33541806, 1.31631048, 1.32914604,
       1.20017431, 1.28866119, 1.28612811, 1.24677689, 1.22806563,
       1.42759683])

In [69]:
np.save('sample_mean_10MHz', sample_mean_10MHz)
np.save('sample_mean_20MHz', sample_mean_20MHz)

In [189]:
### Mardia normality test
def Mardiatest(n, k, Data):
    def gamma(n, Data):
        S = np.cov(Data.T, bias = True)
        gamma_1 = 0; gamma_2 = 0; m_ij = 0
        for i in range(n):
            for j in range(n):
                m_ij = np.matmul(np.matmul((Data[i]- np.mean(Data, axis = 0)).transpose(), inv(S)), (Data[j]-np.mean(Data, axis = 0)))
                gamma_1 += m_ij
                if j == i:
                    gamma_2 += pow(m_ij, 2)

        #print(beta, A, B, C, T)
        return gamma_1/pow(n,2), gamma_2/n      
    gamma1, gamma2 = gamma(n, Data)
    gamma1_df = k*(k+1)*(k+2)/6
    gamma2_mean = k*(k+2)
    gamma2_var = 8*k*(k+2)/6
    pvalue_gamma1 = 1- stats.chi2.cdf(gamma1*n/6, gamma1_df)
    pvalue_gamma2 = 1- stats.norm.cdf(gamma2, loc = gamma2_mean, scale = gamma2_var)
    return np.minimum(pvalue_gamma1, pvalue_gamma2)
    

In [4]:
### multivariate normal test using Henze-Zirkler method

def Tvariable(n, k, Data):
    S = np.cov(Data.T, bias = True)
    beta = (1/np.sqrt(2))*((n*(2*k+1)/4)**(1/(k+4)))
    C = n*(1 + 2*beta**2)**(-k/2)  
    B = 0; T = 0
    for i in range(n):
        A = 0
        for j in range(n):
            A += np.exp(-(beta**2/2)*(np.matmul(np.matmul((Data[i]- Data[j]).transpose(), inv(S)), (Data[i]-Data[j]))))
        B = np.exp(-(beta**2/(2*(1+beta**2)))*(np.matmul(np.matmul((Data[i]- np.mean(Data, axis = 0)).transpose(), inv(S)), (Data[i]-np.mean(Data, axis = 0)))))
        T += (1/n)*A -2*(1 + beta**2)**(-k/2) * B
    return beta, T + C       

def Moments(beta, k):
    w = (1+beta**2)*(1+3*beta**2)
    Expec_T = 1-((1 + 2*beta**2)**(-k/2)*(1 + k*beta**2/(1+2*beta**2) + k*(k+2)*beta**4/2/(1+2*beta**2)**2))
    Var_T = 2*(1 + 4*beta**2)**(-k/2) + 2*((1+2*beta**2)**(-k))*(1+ 2*k*beta**4/(1+2*beta**2)**2 + (3*k*(k+2)*beta**8)/(4*(1+2*beta**2)**4))-(4*w**(-k/2))*(1 + (3*k*beta**4/2/w) + (k*(k+2)*beta**8/2/w**2))
    return Expec_T, Var_T

def jointpval(n, k, Data):
    beta, T = Tvariable(n, k, Data)
    Expec_T, Var_T = Moments(beta, k)
    VZ = np.log(1+ Var_T/(Expec_T**2))
    SD = np.sqrt(VZ)
    EZ = np.log(np.sqrt(Expec_T**4/(Var_T+Expec_T**2)))
    stand_z = (T - EZ)/(np.sqrt(VZ))
    joint_pval = 1-stats.lognorm.cdf(T, s = SD, scale = np.exp(EZ))
    #print(T, Expec_T, Var_T, EZ, SD, stand_z)    
    return joint_pval

In [171]:
!jupyter nbconvert --to script QoSAnalysis_preprocess.ipynb

[NbConvertApp] Converting notebook QoSAnalysis_preprocess.ipynb to script
[NbConvertApp] Writing 12260 bytes to QoSAnalysis_preprocess.py
